## 1. 데이터 로드

In [10]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_0_32 = pd.read_csv("../binutils_gcc3~9_op0~4_csv/"+'gcc6'+"_0_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_0_32.head()

(2358543, 2)
reset_index 완료
input data shape


,hex,bin
0,55,1
1,89,0
2,e5,0
3,56,0
4,53,0


## 2. Data 확인

In [13]:
# DATA 모양 확인
gcc6_0_32.values
print(gcc6_0_32.shape)

# 함수의 갯수
gcc6_0_32['bin'].value_counts()

# 함수 평균길이
gcc6_0_32['bin'].value_counts()[0]/gcc6_0_32['bin'].value_counts()[1]

array([['55', 1],
       ['89', 0],
       ['e5', 0],
       ...,
       ['1c', 0],
       ['24', 0],
       ['c3', 0]], dtype=object)

(2358543, 2)


0    2346253
1      12290
Name: bin, dtype: int64

190.90748576078113

## 3. 함수의 갯수, 바이트 갯수의 빈공간 np.array 생성 

In [14]:
x = np.full((gcc6_0_32['bin'].value_counts()[1],30000), None, dtype='object')
x.shape
x.dtype

(12290, 30000)

dtype('O')

## 4. 함수 빈공간에 함수별로 한줄씩 값 대입

In [15]:
data = gcc6_0_32.values

row = -1
col = 0
col_list=[]

# data => [55,1], [89,1], [e5,1] ....
for i in data:
    # i[0] = 55, 89, e5... /  i[1] = 1, 0, 0, ....
    if i[1]==1:
        col_list.append(col) # col_list의 각 함수 길이들 넣는다.
        
        row+=1
        col=0
        x[row][col] = i[0]
#        print(row,col, end='/') # 오류났을 때 풀어보면 몇번째 행에서 값을 초과하는지 알 수 있음.
    else:
        x[row][col+1] = i[0]
        #if row >=1385:
        #    print(row,col, end=' ')
        col+=1
    
print('대입 완료')    

대입 완료


In [16]:
max(col_list)

21823

## 5. 최적의 x 찾기

In [17]:
# 예시
print('예시')
x[:, :5]
print(len(col_list),'\n')

# 최소 3gram ~ 7gram 까지확인
for j in range(2, 7):
    tuple_ls = []
    for k in x[:, :j]:
        tuple_ls.append(tuple(k))

    set_ls = set(tuple_ls)
    print(j, 'gram일 때 ', len(set_ls))

예시


array([['55', '89', 'e5', '56', '53'],
       ['8b', '1c', '24', 'c3', '55'],
       ['55', '89', 'e5', 'e8', 'fc'],
       ...,
       ['8b', '1c', '24', 'c3', None],
       ['55', '89', 'e5', '53', '83'],
       ['8b', '1c', '24', 'c3', None]], dtype=object)

12290 

2 gram일 때  98
3 gram일 때  763
4 gram일 때  4520
5 gram일 때  4684
6 gram일 때  5131


In [18]:
# 임의 테스트
tls = []
for t in x[:, :4]:
    tls.append(tuple(t))
    
stls = set(tls)
stls

{('c7', '83', '6c', '00'),
 ('8d', '80', '1a', '41'),
 ('8d', '80', '16', '08'),
 ('8d', '80', 'f3', '3c'),
 ('8d', '80', 'e8', '9f'),
 ('8d', '80', 'a3', '4a'),
 ('8d', '80', '93', '56'),
 ('8d', '80', 'e6', '0d'),
 ('8d', '80', 'ec', 'b6'),
 ('8d', '80', 'ab', '8f'),
 ('8d', '80', '91', '2c'),
 ('8d', '80', '85', '0b'),
 ('8d', '80', '4b', '50'),
 ('8d', '80', '23', '57'),
 ('83', '7d', 'b8', '1f'),
 ('8d', '80', '8a', '30'),
 ('8d', '80', '97', '42'),
 ('8d', '80', '34', '15'),
 ('8d', '80', 'e8', '65'),
 ('c7', '45', 'd8', '10'),
 ('8d', '80', '7f', 'c4'),
 ('8d', '80', 'bf', '01'),
 ('8d', '80', '07', '0c'),
 ('8d', '80', 'bd', 'de'),
 ('8d', '80', 'c6', 'cf'),
 ('8d', '80', '42', '8b'),
 ('8d', '80', 'af', 'bc'),
 ('c7', '45', 'e0', '26'),
 ('8d', '80', 'da', '1b'),
 ('8d', '80', 'bb', '9f'),
 ('83', 'bd', '50', 'ff'),
 ('c7', '45', 'e4', '08'),
 ('83', '7d', '10', '00'),
 ('8d', '80', '2f', '99'),
 ('8d', '80', '0f', 'b8'),
 ('8d', '80', '77', 'c0'),
 ('8d', '80', 'a3', '2a'),
 